In [2]:
import boto3, json
import zipfile
import io


In [2]:
# Criar a Função Lambda

lambda_client = boto3.client("lambda", region_name="us-east-1")
iam_client = boto3.client("iam")

# Cria role básica para Lambda
role_name = "LambdaGameExecutionRole"
assume_role_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {"Service": "lambda.amazonaws.com"},
            "Action": "sts:AssumeRole"
        }
    ]
}

try:
    role = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy)
    )
    iam_client.attach_role_policy(
        RoleName=role_name,
        PolicyArn="arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole"
    )
    role_arn = role["Role"]["Arn"]
except iam_client.exceptions.EntityAlreadyExistsException:
    role_arn = iam_client.get_role(RoleName=role_name)["Role"]["Arn"]

# Código simples do jogo
lambda_code = """
import json, random

def lambda_handler(event, context):
    numero_secreto = random.randint(1,10)
    palpite = int(event.get("queryStringParameters", {}).get("numero", 0))
    if palpite == numero_secreto:
        msg = "Parabéns! Você acertou!"
    else:
        msg = f"Errou! O número era {numero_secreto}"
    return {
        "statusCode": 200,
        "body": json.dumps({"mensagem": msg})
    }
"""

# Empacotar código em zip
zip_buffer = io.BytesIO()
with zipfile.ZipFile(zip_buffer, "w") as zf:
    zf.writestr("lambda_function.py", lambda_code)
zip_buffer.seek(0)

# Criar função Lambda
function_name = "LambdaGame-Tigrao"
response = lambda_client.create_function(
    FunctionName=function_name,
    Runtime="python3.9",
    Role=role_arn,
    Handler="lambda_function.lambda_handler",
    Code={"ZipFile": zip_buffer.read()},
    Publish=True
)
print("Lambda criada:", response["FunctionArn"])


NoCredentialsError: Unable to locate credentials

In [ ]:
#Criar API Gateway e integrar com Lambda

apigw_client = boto3.client("apigatewayv2", region_name="us-east-1")

# Criar API HTTP
api = apigw_client.create_api(
    Name="Api-Tigrao",
    ProtocolType="HTTP",
    Target=response["FunctionArn"]
)

api_id = api["ApiId"]

# Criar integração com Lambda
integration = apigw_client.create_integration(
    ApiId=api_id,
    IntegrationType="AWS_PROXY",
    IntegrationUri=response["FunctionArn"],
    PayloadFormatVersion="2.0"
)

# Criar rota /jogo
apigw_client.create_route(
    ApiId=api_id,
    RouteKey="GET /jogo",
    Target=f"integrations/{integration['IntegrationId']}"
)

# Deploy automático (default stage)
apigw_client.create_stage(
    ApiId=api_id,
    StageName="prod",
    AutoDeploy=True
)

print("API criada. Invoke URL:")
print(f"https://{api_id}.execute-api.us-east-1.amazonaws.com/prod/jogo")


In [ ]:
#Criar bucket S3 e hospedar site estático
import json

s3_client = boto3.client("s3", region_name="us-east-1")

bucket_name = "s3-website-tigrao"
s3_client.create_bucket(
    Bucket=bucket_name,
    CreateBucketConfiguration={"LocationConstraint": "us-east-1"}
)

# Upload do index.html (simples apontando para API)
index_html = f"""
<!DOCTYPE html>
<html>
<head><title>Jogo de Adivinhação</title></head>
<body>
  <h1>Jogo de Adivinhação 🎲</h1>
  <input type="number" id="num" min="1" max="10">
  <button onclick="jogar()">Enviar</button>
  <p id="resposta"></p>

  <script>
    async function jogar() {{
      let n = document.getElementById("num").value;
      let url = "https://{api_id}.execute-api.us-east-1.amazonaws.com/prod/jogo?numero=" + n;
      let resp = await fetch(url);
      let data = await resp.json();
      document.getElementById("resposta").innerText = data.mensagem;
    }}
  </script>
</body>
</html>
"""

s3_client.put_object(
    Bucket=bucket_name,
    Key="index.html",
    Body=index_html,
    ContentType="text/html"
)

# Configurar como site estático
s3_client.put_bucket_website(
    Bucket=bucket_name,
    WebsiteConfiguration={
        "IndexDocument": {"Suffix": "index.html"}
    }
)

# Tornar público
policy = {
    "Version": "2012-10-17",
    "Statement": [{
        "Sid": "PublicReadGetObject",
        "Effect": "Allow",
        "Principal": "*",
        "Action": "s3:GetObject",
        "Resource": f"arn:aws:s3:::{bucket_name}/*"
    }]
}
s3_client.put_bucket_policy(
    Bucket=bucket_name,
    Policy=json.dumps(policy)
)

print("Site hospedado no S3:")
print(f"http://{bucket_name}.s3-website-us-east-1.amazonaws.com")
